# Topic modeling para *Big Data*
Vamos a ver cómo realizar un modelado de temática en grandes volúmenes de texto con la librería `gensim`  

Utilizaremos el conjunto de datos *Lee* de `Gensim` (es una versión abreviada del conjunto http://www.socsci.uci.edu/~mdlee/lee_pincombe_welsh_document.PDF).  

Para visualizar gráficamente los tópicos es necesario instalar la librería `pyLDAvis` dentro del entorno de Anaconda con el comando:
```python
conda install -c conda-forge pyldavis 
```

### Cargamos librerías

In [2]:
import os
import re
import numpy as np
import pandas as pd
from pprint import pprint
import warnings

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
warnings.filterwarnings('ignore')

# spacy para lematizar
import spacy

# herramientas de dibujado
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline

/home/vic_263/anaconda3/envs/ia/lib/python3.10/site-packages/IPython/core/pylabtools.py:77: DeprecationWarning: backends is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.
  warnings.warn(
/home/vic_263/anaconda3/envs/ia/lib/python3.10/site-packages/IPython/core/pylabtools.py:77: DeprecationWarning: backend2gui is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.
  warnings.warn(


Utilizamos un generador para obtener los documentos del Corpus línea a línea desde el archivo del conjunto de ejemplo y convertirlos en un listado de tokens.

In [3]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
stop_words = nlp.Defaults.stop_words #listado de stop-words

def lemmatize_corpus(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
    """Función que devuelve el lema de una string,
    excluyendo las palabras cuyo POS_TAG no está en la lista"""
    text_out = [t.lemma_.lower() for t in nlp(text)
                if t.pos_ in allowed_postags
                and len(t.lemma_)>3
                and not t.is_stop]
    return text_out
            
def build_texts(fname):
    """
    Generador que devuelve el texto tokenizado a partir de un archivo
    línea a línea
    """
    with open(fname) as f:
        for line in f:
            yield lemmatize_corpus(line)

In [4]:
data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_data_file = data_dir + os.sep + 'lee_background.cor'

In [5]:
lee_data_file

'/home/vic_263/anaconda3/envs/ia/lib/python3.10/site-packages/gensim/test/test_data/lee_background.cor'

In [6]:
texto=build_texts(lee_data_file)

In [7]:
texto

<generator object build_texts at 0x7fa1ccdc8ba0>

In [8]:
print(next(texto))

['hundred', 'people', 'force', 'vacate', 'home', 'southern', 'highlands', 'south', 'wales', 'strong', 'wind', 'today', 'push', 'huge', 'bushfire', 'town', 'hill', 'blaze', 'goulburn', 'south', 'west', 'sydney', 'force', 'closure', 'hume', 'highway', 'aedt', 'marked', 'deterioration', 'weather', 'storm', 'cell', 'move', 'east', 'blue', 'mountains', 'force', 'authority', 'decision', 'evacuate', 'people', 'home', 'outlying', 'street', 'hill', 'south', 'wales', 'southern', 'highland', 'estimate', 'resident', 'leave', 'home', 'nearby', 'mittagong', 'south', 'wales', 'rural', 'fire', 'service', 'weather', 'condition', 'cause', 'fire', 'burn', 'finger', 'formation', 'ease', 'fire', 'unit', 'hill', 'optimistic', 'defend', 'property', 'blaze', 'burn', 'year', 'south', 'wales', 'fire', 'crew', 'call', 'fire', 'gunning', 'south', 'goulburn', 'detail', 'available', 'stage', 'fire', 'authority', 'close', 'hume', 'highway', 'direction', 'fire', 'sydney', 'west', 'long', 'threaten', 'property', 'cran

In [9]:
for t in texto:
    print(t)
    break

['indian', 'security', 'force', 'shoot', 'dead', 'suspect', 'militant', 'night', 'long', 'encounter', 'southern', 'kashmir', 'shootout', 'take', 'place', 'dora', 'village', 'kilometer', 'south', 'kashmiri', 'summer', 'capital', 'srinagar', 'death', 'come', 'pakistani', 'police', 'arrest', 'dozen', 'militant', 'extremist', 'group', 'accuse', 'stage', 'attack', 'india', 'parliament', 'india', 'accuse', 'pakistan', 'base', 'lashkar', 'taiba', 'jaish', 'mohammad', 'carry', 'attack', 'december', 'behest', 'pakistani', 'military', 'intelligence', 'military', 'tension', 'soar', 'raid', 'side', 'mass', 'troop', 'border', 'trade', 'diplomatic', 'sanction', 'yesterday', 'pakistan', 'announce', 'arrest', 'lashkar', 'taiba', 'chief', 'hafiz', 'mohammed', 'saeed', 'police', 'karachi', 'likely', 'raid', 'launch', 'group', 'militant', 'organisation', 'accuse', 'targette', 'india', 'military', 'tension', 'india', 'pakistan', 'escalate', 'level']


### Creamos el diccionario y el corpus para Topic Modeling
Las dos entradas para el modelo LDA son un diccionario de `gensim` y un corpus de texto.  
Preparamos el diccionario:

In [10]:
class BOW_Corpus(object):
    """
    Iterable: en cada iteración devuelve el vector bag-of-words
    del siguiente documento en el corpus.
    El corpus es el listado de críticas alojadas en el directorio
    pasado como argumento al instanciar la clase.
    
    Procesa un documento cada vez, así
    nunca carga el corpus entero en RAM.
    """
    def __init__(self, filename):
        self.filename = filename
        #creamos bigramas y trigramas
        self.bigram = gensim.models.Phrases(build_texts(self.filename), min_count=5, threshold=50) # higher threshold fewer phrases.
        #optimizamos una vez entreando
        self.bigram_mod = gensim.models.phrases.Phraser(self.bigram)

        self.trigram = gensim.models.Phrases(self.bigram_mod[build_texts(self.filename)], min_count=5, threshold=50)  
        self.trigram_mod = gensim.models.phrases.Phraser(self.trigram)
        #crea el diccionario = mapeo de documentos a sparse vectors
        self.diccionario = gensim.corpora.Dictionary(self.trigram_mod[map(lambda x: self.bigram_mod[x], build_texts(self.filename))])

    def __len__(self):
        #necesitamos saber la longitud del corpus para visualizar con pyLDAvis
        return self.diccionario.num_docs
    
    def __iter__(self):
        """
        __iter__ es un iterable => BOW_Corpus es un streamed iterable.
        """
        for tokens in build_texts(self.filename):
            # transforma cada doc (lista de tokens) en un vector sparse uno a uno
            yield self.diccionario.doc2bow(self.trigram_mod[self.bigram_mod[tokens]])

In [11]:
corpus_bow = BOW_Corpus(lee_data_file)

In [12]:
len(corpus_bow)

300

In [13]:
corpus_bow.diccionario.num_docs

300

In [14]:
len(corpus_bow.diccionario.token2id)

5136

In [15]:
[k for k in corpus_bow.diccionario.token2id if re.match(r'\w+_\w+_\w+', k)]

['rural_fire_service',
 'leader_yasser_arafat',
 'industrial_relations_commission',
 'foreign_minister_alexander_downer',
 'president_george_bush',
 'woomera_detention_centre',
 'hamas_islamic_jihad']

In [16]:
len([k for k in corpus_bow.diccionario.token2id if re.match(r'\w+_\w+', k)])

93

In [17]:
for c in corpus_bow:
    print(c)
    print(len(c))
    break

[(0, 1), (1, 2), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 1), (8, 3), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 3), (33, 1), (34, 7), (35, 1), (36, 3), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 3), (47, 3), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 2), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 1), (69, 1), (70, 2), (71, 1), (72, 3), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 4), (80, 2), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 3), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 2), (95, 2), (96, 2), (97, 1)]
98


### Modelo LDA
Es un modelo generativo que considera cada documento como una mezcla de temas donde cada tema tiene una distribución de las palabras.

In [18]:
warnings.filterwarnings('ignore')


ldamodel = LdaModel(corpus=corpus_bow, num_topics=10, id2word=corpus_bow.diccionario)
pprint(ldamodel.print_topics())

[(0,
  '0.012*"palestinian" + 0.011*"people" + 0.009*"israeli" + 0.006*"report" + '
  '0.006*"government" + 0.005*"fire" + 0.005*"arafat" + 0.005*"security" + '
  '0.005*"area" + 0.005*"attack"'),
 (1,
  '0.009*"australia" + 0.007*"year" + 0.006*"government" + 0.006*"claim" + '
  '0.005*"force" + 0.005*"union" + 0.005*"attack" + 0.004*"people" + '
  '0.004*"plan" + 0.004*"security"'),
 (2,
  '0.011*"palestinian" + 0.006*"group" + 0.006*"fire" + 0.006*"attack" + '
  '0.005*"arrest" + 0.005*"government" + 0.005*"year" + 0.005*"israeli" + '
  '0.005*"people" + 0.005*"kill"'),
 (3,
  '0.007*"australia" + 0.007*"year" + 0.006*"test" + 0.006*"force" + '
  '0.006*"australian" + 0.006*"israeli" + 0.006*"palestinian" + 0.004*"time" + '
  '0.004*"government" + 0.003*"kill"'),
 (4,
  '0.008*"year" + 0.007*"official" + 0.006*"australia" + 0.005*"fire" + '
  '0.004*"attack" + 0.004*"area" + 0.004*"people" + 0.004*"palestinian" + '
  '0.004*"think" + 0.003*"come"'),
 (5,
  '0.007*"people" + 0.006*"a

### Visualización de los temas  
Podemos visualizarlo gráficamente la distribución de los documentos del Corpus por temas con la librería `pyLDAvis`

In [19]:
warnings.filterwarnings('ignore')
vis_data = gensimvis.prepare(ldamodel, corpus_bow, corpus_bow.diccionario)
pyLDAvis.display(vis_data)

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
for c in corpus_bow:
    print(c)
    break

In [ ]:
#reducimos el vocabulario
corpus_bow.diccionario.filter_extremes(no_above=0.7)

In [ ]:
len(corpus_bow.diccionario.token2id)

In [ ]:
for c in corpus_bow:
    print(len(c))
    break

In [ ]:
warnings.filterwarnings('ignore')
ldamodel = LdaModel(corpus=corpus_bow, num_topics=10, id2word=corpus_bow.diccionario)
vis_data = gensimvis.prepare(ldamodel, corpus_bow, corpus_bow.diccionario)
pyLDAvis.display(vis_data)

In [ ]:
for l in ldamodel[corpus_bow]:
    print(l)
    break

In [ ]:
ldamodel[corpus_bow]